In [55]:
import json
import numpy as np
from matplotlib.pyplot import plot
import matplotlib.pyplot as plt

with open(r'C:\Users\Guo\Downloads\WatchData (23).json') as w:
    data = json.load(w)
window=32
channel=18
DATA=np.empty((0,window,channel))
DATA_norm=np.empty((0,window,channel))

def scale(X):
    channel = (X.shape)[-1]
    for i in range(channel):
        mean = np.mean(X[:,:,i])
        std  = np.std(X[:,:,i])
        X[:,:,i]=np.tanh((X[:,:,i]-mean)/std)
    
    return X
def interp(xp,fp):
    x=np.zeros(shape=(int(xp[-1]/20)+1,13), dtype=float)
    new_time=np.arange(0,xp[-1],20)
    for i in range(13):
        x[:,i]=np.interp(new_time,xp,fp[:,i])
    return new_time,x
data_counter=0
gyro_x=[]
gyro_y=[]
gyro_z=[]
gyro_time=[]
lin_x=[]
lin_y=[]
lin_z=[]
lin_time=[]
grav_x=[]
grav_y=[]
grav_z=[]
grav_time=[]
for dat in data:
    sdata=dat['sensor_data']
    n=len(sdata)
    
    for i in range(0,n,1):
        if sdata[i]['s'] ==4:
            x=sdata[i]['d']['x']
            y=sdata[i]['d']['y']
            z=sdata[i]['d']['z']
            time=sdata[i]['ts']

            gyro_x.append(x)
            gyro_y.append(y)
            gyro_z.append(z)
            gyro_time.append(np.rint(time/1000000))
        if sdata[i]['s'] == 10:
            x=sdata[i]['d']['x']
            y=sdata[i]['d']['y']
            z=sdata[i]['d']['z']
            time=sdata[i]['ts']

            lin_x.append(x)
            lin_y.append(y)
            lin_z.append(z)
            lin_time.append(np.rint(time/1000000))
        if sdata[i]['s'] == 9:
            x=sdata[i]['d']['x']
            y=sdata[i]['d']['y']
            z=sdata[i]['d']['z']
            time=sdata[i]['ts']

            grav_x.append(x)
            grav_y.append(y)
            grav_z.append(z)
            grav_time.append(np.rint(time/1000000))
        
    data_counter=data_counter+n
print(data_counter)
    
    
len_array=[len(gyro_time), len(lin_time), len(grav_time)]
length=min(len_array)
print(length)
print(len_array)

83895
20972
[20972, 20974, 20974]


In [56]:
len_new=0
gyrox=[]
gyroy=[]
gyroz=[]

linx=[]
liny=[]
linz=[]

gravx=[]
gravy=[]
gravz=[]

validTime=[]

for linind in range(0, len(lin_time),1):
    time=lin_time[linind]

    if (time in gyro_time) & (time in grav_time):

        gyroind=gyro_time.index(time)
        gravind=grav_time.index(time)
        

        gyrox.append(gyro_x[gyroind])
        gyroy.append(gyro_y[gyroind])
        gyroz.append(gyro_z[gyroind])

        linx.append(lin_x[linind])
        liny.append(lin_y[linind])
        linz.append(lin_z[linind])

        gravx.append(grav_x[gravind])
        gravy.append(grav_y[gravind])
        gravz.append(grav_z[gravind])

        validTime.append(time)

        len_new+=1
    #else:
        #print(linind, time)

start = validTime[0]
for i in range(len(validTime)):
    validTime[i] = (validTime[i]-start)

X=np.zeros(shape=(len_new,9), dtype=float)
X[:,0]=np.asarray(gyrox[:len_new])
X[:,1]=np.asarray(gyroy[:len_new])
X[:,2]=np.asarray(gyroz[:len_new])
X[:,3]=np.asarray(linx[:len_new])
X[:,4]=np.asarray(liny[:len_new])
X[:,5]=np.asarray(linz[:len_new])
X[:,6]=np.asarray(gravx[:len_new])
X[:,7]=np.asarray(gravy[:len_new])
X[:,8]=np.asarray(gravz[:len_new])

X.shape


(20972, 9)

In [57]:

#validTime,X=interp(validTime,X)

np.save('./watch_4_29_raw.npy',X)
X_time=np.append(np.asarray(validTime).reshape(len(X),1),X,axis=1)
np.savetxt('./watch_4_29_raw.csv',X_time,delimiter=",")
len_new=len(validTime)

XX=np.empty(shape=(len_new-window+1, window, channel))
counter=0
uncounter=0

for i in range(0,len_new-window+1,1):
    if(validTime[i+window-1]-validTime[i]<42*window):
        XX[counter,:, :9] = X[i : i+window, :]
        counter+=1
    else:
        uncounter+=1
        print(validTime[i])
print("counter: "+str(counter))
print("unvalid:"+str(uncounter))

DATA=np.append(DATA,XX[:counter,:,:],axis=0)
np.save('./watch_4_29.npy',DATA)
DATA_norm=scale(DATA)

n_sample=DATA_norm.shape[0]

for j in range(0,n_sample,1):
    for k in range(0,9,1):
        fft=np.fft.fft(DATA_norm[j,:,k])
        DATA_norm[j,:,k+9]=abs(fft)
        
DATA_norm[:,:,9:18]=scale(DATA_norm[:,:,9:18])
np.save('./watch_norm_128_4_29_.npy', DATA_norm)

counter: 20941
unvalid:0


C:\Users\Guo\Anaconda3\envs\Tensorflow_cpu\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
